In [ ]:
# Download a few dataset in yolo11 format from roboflow-universe-projects

from pathlib import Path

# final class list for training (edit as needed)
FINAL_CLASSES = ["license_plate", "weapon"]
final_to_id = {c:i for i,c in enumerate(FINAL_CLASSES)}

# point to your downloaded/unzipped dataset roots (each contains a data.yml)
RAW_DATASETS = [
    Path("/home/ec2-user/self_train_yolo11/fire_arm_detection.v1i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/License_Plate_Recognition.v11i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/weapon_detection.v10i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/e_commerce_gun_detection.v5i.yolov11"),
]
    
# where to build the merged dataset
MERGED_ROOT = Path("/home/ec2-user/self_train_yolo11").resolve()
(MERGED_ROOT / "images" / "train").mkdir(parents=True, exist_ok=True)
(MERGED_ROOT / "images" / "val").mkdir(parents=True, exist_ok=True)
(MERGED_ROOT / "images" / "test").mkdir(parents=True, exist_ok=True)
(MERGED_ROOT / "labels" / "train").mkdir(parents=True, exist_ok=True)
(MERGED_ROOT / "labels" / "val").mkdir(parents=True, exist_ok=True)
(MERGED_ROOT / "labels" / "test").mkdir(parents=True, exist_ok=True)

MERGED_ROOT



In [ ]:
# Read project's data.yaml
import yaml

def read_dataset_names(ds_root: Path):
    y = yaml.safe_load((ds_root / "data.yaml").read_text())
    # Roboflow often uses 'val' in YAML but folder is 'valid'. We’ll normalize later.
    names_field = y.get("names")
    if isinstance(names_field, dict):  # {0:'name0', 1:'name1', ...}
        idx_to_name = [names_field[i] for i in sorted(names_field.keys())]
    else:  # ['name0','name1',...]
        idx_to_name = names_field
    return idx_to_name

for ds in RAW_DATASETS:
    print(ds, read_dataset_names(ds))

In [ ]:
#  Define mapping rules per dataset
PER_DATASET_CLASS_MAP = {
    "/home/ec2-user/self_train_yolo11/fire_arm_detection.v1i.yolov11": {
        "we - v2 2024-10-09 10-14am": "weapon",
    },
    "/home/ec2-user/self_train_yolo11/License_Plate_Recognition.v11i.yolov11": {
        "License_Plate": "license_plate"
    },
    "/home/ec2-user/self_train_yolo11/weapon_detection.v10i.yolov11": {
        "gun": "weapon",
        "knife": "weapon",
        # everything else ignored
        "arm": None,
        "body": None,
        "face": None,
        "foot": None,
        "hand": None,
        "human": None,
        "leg": None,
        "object": None,
        "person": None,
    }, 
    "/home/ec2-user/self_train_yolo11/e_commerce_gun_detection.v5i.yolov11": {
        "weapon": "weapon"
    },
}

In [ ]:
# Merge datasets
import shutil
from tqdm import tqdm

def iter_split_dirs(ds_root: Path):
    # Some zips use 'valid', some 'val'. Normalize to ('train','val','test').
    train = ds_root / "train"
    val   = ds_root / "val"
    valid = ds_root / "valid"
    test  = ds_root / "test"

    if valid.exists() and not val.exists():
        val = valid

    return {
        "train": train if train.exists() else None,
        "val":   val   if val.exists()   else None,
        "test":  test  if test.exists()  else None,
    }

def remap_and_copy_split(ds_root: Path, split: str, mapping: dict):
    # mapping: original_name -> final_name (or None to drop)
    src_images = ds_root / split / "images"
    src_labels = ds_root / split / "labels"
    if not src_images.exists() or not src_labels.exists():
        return 0,0

    dst_images = MERGED_ROOT / "images" / split
    dst_labels = MERGED_ROOT / "labels" / split

    # build name->id for the source dataset
    src_names = read_dataset_names(ds_root)
    src_id_to_name = dict(enumerate(src_names))

    copied_images = 0
    kept_boxes = 0

    for img_path in tqdm(list(src_images.glob("*.*")), desc=f"{ds_root.name}:{split}"):
        label_path = src_labels / (img_path.stem + ".txt")
        if not label_path.exists():
            # copy image with empty label? Usually skip to keep consistency.
            continue

        # read & remap label lines
        new_lines = []
        for line in label_path.read_text().splitlines():
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            cid = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:])
            original_name = src_id_to_name.get(cid, None)
            if original_name is None:
                continue
            final_name = mapping.get(original_name, None)
            if final_name is None:
                continue  # dropped
            new_cid = final_to_id[final_name]
            new_lines.append(f"{new_cid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        if not new_lines:
            # All boxes dropped (e.g., dataset had only 'weapon' and mapping dropped it)
            continue

        # copy image + write remapped label
        shutil.copy2(img_path, dst_images / img_path.name)
        (dst_labels / f"{img_path.stem}.txt").write_text("\n".join(new_lines), encoding="utf-8")
        copied_images += 1
        kept_boxes += len(new_lines)

    return copied_images, kept_boxes

total_imgs = total_boxes = 0
for ds in RAW_DATASETS:
    maps = PER_DATASET_CLASS_MAP.get(str(ds), None)
    if maps is None:
        print(f"⚠️ No mapping for {ds}. Skipping.")
        continue
    splits = iter_split_dirs(ds)
    for sp, sp_dir in splits.items():
        if sp_dir is None: 
            continue
        ci, kb = remap_and_copy_split(ds, sp, maps)
        total_imgs += ci
        total_boxes += kb
        print(f"{ds.name}:{sp} → images:{ci}, boxes:{kb}")

print(f"MERGED totals → images:{total_imgs}, boxes:{total_boxes}")


In [ ]:
import yaml

merged_data = {
    "path": str(MERGED_ROOT),
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "names": {i:cls for i,cls in enumerate(FINAL_CLASSES)}
}
with open(MERGED_ROOT / "data.yaml", "w", encoding="utf-8") as f:
    yaml.safe_dump(merged_data, f, allow_unicode=True)

print((MERGED_ROOT / "data.yaml").read_text())

In [ ]:
import pandas as pd
from pathlib import Path

def scan_labels(root: Path, split: str):
    rows = []
    for p in (root / "labels" / split).glob("*.txt"):
        for ln in p.read_text().splitlines():
            parts = ln.split()
            if len(parts)!=5: 
                continue
            cid = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:])
            rows.append((split, p.stem, cid, xc, yc, w, h))
    return rows

rows = []
for sp in ["train","val","test"]:
    rows += scan_labels(MERGED_ROOT, sp)

df = pd.DataFrame(rows, columns=["split","image_id","cid","xc","yc","w","h"])
print("Label rows:", len(df))
print(df["cid"].value_counts().sort_index(), "cid counts (0..n)")

# quick class histogram by split
print(df.groupby(["split","cid"]).size())


In [ ]:
import pandas as pd
from pathlib import Path

def scan_labels(root: Path, split: str):
    rows = []
    for p in (root / "labels" / split).glob("*.txt"):
        for ln in p.read_text().splitlines():
            parts = ln.split()
            if len(parts)!=5: 
                continue
            cid = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:])
            rows.append((split, p.stem, cid, xc, yc, w, h))
    return rows

rows = []
for sp in ["train","val","test"]:
    rows += scan_labels(MERGED_ROOT, sp)

df = pd.DataFrame(rows, columns=["split","image_id","cid","xc","yc","w","h"])
print("Label rows:", len(df))
print(df["cid"].value_counts().sort_index(), "cid counts (0..n)")

# quick class histogram by split
print(df.groupby(["split","cid"]).size())


In [ ]:
# 1) 參數 & 路徑
import shutil
from tqdm import tqdm

MERGED_ROOT = Path("/home/ec2-user/self_train_yolo11")
RAW = [
    Path("/home/ec2-user/self_train_yolo11/fire_arm_detection.v1i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/License_Plate_Recognition.v11i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/weapon_detection.v10i.yolov11"),
    Path("/home/ec2-user/self_train_yolo11/e_commerce_gun_detection.v5i.yolov11"),
]

#FINAL_CLASSES = ["license_plate", "weapon"]
#final_to_id = {c:i for i,c in enumerate(FINAL_CLASSES)}

# 👇請依你各資料集 data.yaml 真實 names 來填（大小寫/空白要完全一致）
PER_DATASET_CLASS_MAP = {
    str(RAW[0]): {  # fire_arm_detection.v1i.yolov11
        "we - v2 2024-10-09 10-14am": "weapon",
    },
    str(RAW[1]): {  # License_Plate_Recognition.v11i.yolov11
        "License_Plate": "license_plate",
    },
    str(RAW[2]): {  # weapon_detection.v10i.yolov11
        "gun": "weapon",
        "knife": "weapon",
        "arm": None, "body": None, "face": None, "foot": None, "hand": None,
        "human": None, "leg": None, "object": None, "person": None,
    },
    str(RAW[3]): {  # e_commerce_gun_detection.v5i.yolov11
        "weapon": "weapon",
    },
}

# 2) 讀 names 的小工具
def read_dataset_names(ds_root: Path):
    for fname in ["data.yml", "data.yaml"]:
        p = ds_root / fname
        if p.exists():
            y = yaml.safe_load(p.read_text())
            names = y["names"]
            if isinstance(names, dict):
                return [names[i] for i in sorted(names)]
            return list(names)
    raise FileNotFoundError(f"No data.yml/.yaml in {ds_root}")
# 3) 若 valid 是 segmentation，將 polygon → bbox
def seg_line_to_bbox(parts):
    xs = [float(parts[i]) for i in range(1, len(parts), 2)]
    ys = [float(parts[i]) for i in range(2, len(parts), 2)]
    if not xs or not ys: 
        return None
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    w, h = (xmax - xmin), (ymax - ymin)
    if w <= 0 or h <= 0: 
        return None
    xc = (xmin + xmax) / 2.0
    yc = (ymin + ymax) / 2.0
    return xc, yc, w, h
# 4) 先備份目前 val，再清空
val_img = MERGED_ROOT / "images" / "val"
val_lbl = MERGED_ROOT / "labels" / "val"
bk = MERGED_ROOT / "val_backup_before_remap"
(bk / "images").mkdir(parents=True, exist_ok=True)
(bk / "labels").mkdir(parents=True, exist_ok=True)
for p in val_img.glob("*.*"): shutil.move(str(p), str((bk/"images")/p.name))
for p in val_lbl.glob("*.txt"): shutil.move(str(p), str((bk/"labels")/p.name))
print("Backed up old val to", bk)
val_img.mkdir(parents=True, exist_ok=True)
val_lbl.mkdir(parents=True, exist_ok=True)

# 5) 針對每個 RAW dataset，把 valid → remap → merged/val
copied = kept = 0
for ds in RAW:
    names = read_dataset_names(ds)
    id2name = dict(enumerate(names))
    cmap = PER_DATASET_CLASS_MAP.get(str(ds), {})
    src_img = ds / "valid" / "images"
    src_lbl = ds / "valid" / "labels"
    if not (src_img.exists() and src_lbl.exists()):
        print("Skip", ds.name, ": no valid split")
        continue

    for ip in tqdm(list(src_img.glob("*.*")), desc=f"{ds.name}: valid→val"):
        lp = src_lbl / f"{ip.stem}.txt"
        if not lp.exists():
            continue
        new_lines = []
        for raw in lp.read_text().splitlines():
            parts = raw.strip().split()
            if len(parts) < 5: 
                continue
            try:
                cid = int(float(parts[0]))
            except:
                continue
            src_name = id2name.get(cid)
            tgt = cmap.get(src_name, None)
            if tgt is None or tgt not in final_to_id:
                continue
            new_cid = final_to_id[tgt]
            if len(parts) == 5:
                # detect: cid xc yc w h
                try:
                    xc, yc, w, h = map(float, parts[1:])
                except:
                    continue
            else:
                # segment → bbox
                box = seg_line_to_bbox(parts)
                if not box:
                    continue
                xc, yc, w, h = box
            if w <= 0 or h <= 0:
                continue
            new_lines.append(f"{new_cid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        if not new_lines:
            # 你要保留 val 的純背景嗎？若要，寫入空檔；若不要，直接跳過
            # (val_lbl / f"{ip.stem}.txt").write_text("", encoding="utf-8")
            # shutil.copy2(ip, val_img / ip.name); copied += 1
            continue

        shutil.copy2(ip, val_img / ip.name)
        (val_lbl / f"{ip.stem}.txt").write_text("\n".join(new_lines), encoding="utf-8")
        copied += 1
        kept += len(new_lines)

print(f"VAL rebuilt → images:{copied}, boxes:{kept}")

# 6) 驗證 val 只剩 0/1
counts = {}
for p in val_lbl.glob("*.txt"):
    for ln in p.read_text().splitlines():
        cid = int(float(ln.split()[0]))
        counts[cid] = counts.get(cid, 0) + 1
print("val cid counts:", counts)   # 期待只看到 {0:..., 1:...}



In [ ]:
import os, gc, torch
gc.collect(); torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # reduce fragmentation

from ultralytics import YOLO

model = YOLO("yolo11n.pt")  # smaller than m; try yolo11s.pt if you can
results = model.train(
    data="/home/ec2-user/self_train_yolo11/data.yaml",
    imgsz=960,      # down from 1280; try 960 or 896 if needed
    batch=-1,        # auto-find the largest batch that fits in VRAM
    epochs=80,
    cos_lr=True,
    workers=2,       # smaller dataloader footprint
    amp=True,        # mixed precision (default True; keep it)
    cache=None,      # make sure we don't cache dataset in RAM/GPU
)

In [ ]:
from pathlib import Path

val_lbl = Path("/home/ec2-user/self_train_yolo11/labels/val")
counts = {}
for p in val_lbl.glob("*.txt"):
    for ln in p.read_text().splitlines():
        ln = ln.strip()
        if not ln: 
            continue
        cid = int(float(ln.split()[0]))
        counts[cid] = counts.get(cid, 0) + 1
print(counts) 

In [ ]:
m = YOLO("runs/detect/train4/weights/best.pt")
m.val(data="/home/ec2-user/self_train_yolo11/data.yaml", imgsz=960)

In [ ]:
model = YOLO("runs/detect/train4/weights/best.pt")

model.train(
    data="/home/ec2-user/self_train_yolo11/data.yaml",
    imgsz=1024,
    epochs=10,
    lr0=0.002,
    batch=-1,       # ✅ auto select max batch size
    cos_lr=True,
    workers=2,
)

In [ ]:
from ultralytics import YOLO
model = YOLO("runs/detect/train5/weights/best.pt")  # 比 n 稍大、通常 mAP 明顯更好
model.train(
    data="/home/ec2-user/self_train_yolo11/data.yaml",
    imgsz=960,
    epochs=15,          # fewer epochs; you can extend later if needed
    batch=-1,           # auto-batch once; if it picks >16, cap at 12–16 for stability
    workers=4,          # try 4; if RAM/IO thrash, fall back to 2
    cache="ram",        # cache images in RAM (32GB instance is fine) → big dataloader speedup
    amp=True,           # keep mixed precision
    val=False,          # skip per-epoch val (big time saver) — run .val() after training
    cos_lr=True,
    project="runs/detect",
    name="train5_fast",
    exist_ok=False
)

In [ ]:
!nvidia-smi